<a href="https://colab.research.google.com/github/dave-shore/beer/blob/main/initial_trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [2]:
model_name = "Babelscape/wikineural-multilingual-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [3]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [4]:
labelset = list(model.config.id2label.values())
labelset

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
ex = """Gibilterra (in inglese e spagnolo Gibraltar; pronuncia inglese: /dʒɨˈbɹɒltə/; pronuncia spagnola: /xiβɾalˈtaɾ/) è un territorio d'oltremare del Regno Unito. In quanto tale, è inserito nella lista delle Nazioni Unite dei territori non autonomi.

Si trova nell'Europa sudoccidentale, precisamente nel sud della penisola Iberica, all'estremo orientale dello stretto omonimo che collega l'oceano Atlantico e il mar Mediterraneo. A nord confina interamente con La Línea de la Concepción, comune spagnolo in provincia di Cadice (Andalusia). Gibilterra viene considerato dalla Spagna come una vera e propria colonia in territorio europeo. Dal 2021 fa parte dello spazio Schengen[3].

Un tempo nota come Calpe, una delle Colonne d'Ercole, deve il suo nome alla corruzione del toponimo arabo Jabal Ţāriq (جبل طارق, ossia Monte di Tariq), così chiamato in omaggio a Tariq ibn Ziyad, il condottiero berbero che conquistò la Spagna nel 711[4]."""



In [6]:
encoded_input = tokenizer(ex, return_tensors = "pt")

In [7]:
y = model(**encoded_input, output_hidden_states = True)

In [8]:
y.logits.shape

torch.Size([1, 235, 9])

In [9]:
[(tok, labelset[p.argmax().item()]) for tok,p in zip(encoded_input.encodings[0].tokens, y.logits[0])]

[('[CLS]', 'O'),
 ('G', 'B-LOC'),
 ('##ibil', 'I-LOC'),
 ('##terra', 'I-LOC'),
 ('(', 'O'),
 ('in', 'O'),
 ('inglese', 'O'),
 ('e', 'O'),
 ('spagnolo', 'O'),
 ('Gibraltar', 'B-LOC'),
 (';', 'O'),
 ('pronuncia', 'O'),
 ('inglese', 'O'),
 (':', 'O'),
 ('/', 'O'),
 ('[UNK]', 'O'),
 ('/', 'O'),
 (';', 'O'),
 ('pronuncia', 'O'),
 ('spagnola', 'O'),
 (':', 'O'),
 ('/', 'O'),
 ('x', 'O'),
 ('##i', 'O'),
 ('##β', 'O'),
 ('##ɾ', 'O'),
 ('##al', 'O'),
 ('##ˈ', 'O'),
 ('##ta', 'O'),
 ('##ɾ', 'O'),
 ('/', 'O'),
 (')', 'O'),
 ('è', 'O'),
 ('un', 'O'),
 ('territorio', 'O'),
 ('d', 'O'),
 ("'", 'O'),
 ('oltre', 'O'),
 ('##mare', 'O'),
 ('del', 'O'),
 ('Regno', 'B-LOC'),
 ('Unito', 'I-LOC'),
 ('.', 'O'),
 ('In', 'O'),
 ('quanto', 'O'),
 ('tale', 'O'),
 (',', 'O'),
 ('è', 'O'),
 ('inserito', 'O'),
 ('nella', 'O'),
 ('lista', 'O'),
 ('delle', 'O'),
 ('Nazi', 'B-ORG'),
 ('##oni', 'I-ORG'),
 ('Unite', 'I-ORG'),
 ('dei', 'O'),
 ('territori', 'O'),
 ('non', 'O'),
 ('auto', 'O'),
 ('##nomi', 'O'),
 ('.', 'O'

In [10]:
len(y.hidden_states), [h.shape for h in y.hidden_states]

(13,
 [torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768]),
  torch.Size([1, 235, 768])])

In [11]:
first_ent = y.hidden_states[0][0][1:4]
second_ent = y.hidden_states[0][0][9:10]

In [12]:
context = y.hidden_states[0][0]

In [13]:
!pip install parametric_umap &> /dev/null

In [14]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from parametric_umap import ParametricUMAP
import torch

In [15]:
n_neighbors = 15

In [16]:
umap = ParametricUMAP(n_neighbors = n_neighbors, hidden_dim = (context.shape[-1] + 3) // 2, n_components = 3, n_layers = int(context.shape[0] / context.shape[1] * n_neighbors) + 1)
points3d = umap.fit_transform(context.detach().clone())

Sampling negative edges...


Completed processes:   0%|          | 0/2 [00:00<?, ?it/s]

Training...


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/725 [00:00<?, ?it/s]

...

Epoch 9:   0%|          | 0/725 [00:00<?, ?it/s]

Epoch 9/10, Loss: 0.1512


Epoch 10:   0%|          | 0/725 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.1506


/usr/local/lib/python3.12/dist-packages/parametric_umap/core.py:268: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32).to(self.device)


In [17]:
gf = px.scatter_3d(data_frame = pd.DataFrame(points3d).reset_index(), x=0,y=1,z=2, color = "index")
gf.show()

In [18]:
umap_rd = ParametricUMAP(n_neighbors = n_neighbors, hidden_dim = (context.shape[-1] + 384) // 2, n_components = 384, n_layers = int(context.shape[0] / context.shape[1] * n_neighbors) + 1)
umap_rd.fit(context.detach().clone())


Sampling negative edges...


Completed processes:   0%|          | 0/2 [00:00<?, ?it/s]

Training...


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/725 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 0.1455


Epoch 9:   0%|          | 0/725 [00:00<?, ?it/s]

Epoch 9/10, Loss: 0.1428


Epoch 10:   0%|          | 0/725 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.1405


In [19]:
first_ent_reduced = umap_rd.transform(first_ent)
second_ent_reduced = umap_rd.transform(second_ent)


/usr/local/lib/python3.12/dist-packages/parametric_umap/core.py:268: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [20]:
gf = px.scatter_3d(
    data_frame = pd.DataFrame(torch.cat([first_ent, second_ent]).detach().cpu()).join(pd.DataFrame({"ent": [1]*len(first_ent) + [2]*len(second_ent)})),
    x=0,y=1,z=2, color = "ent")
gf.show()

In [21]:
from transformers import pipeline
from string import punctuation
from itertools import combinations

In [22]:
pipe = pipeline("ner", model = model_name)
ents = pipe(ex)

Device set to use cuda:0


In [23]:
def recompose_entities(records):

  entities = []
  current = ""
  current_type = ""
  current_start = -1
  current_end = -1

  for rec in records:
    if rec['entity'].startswith("B"):
      if current:
        entities.append({"word": current, "entity": current_type, "start": current_start, "end": current_end})
      current = rec['word']
      current_type = rec['entity'].split("-")[-1]
      current_start = rec['index']
      current_end = rec['index'] + 1
    elif rec['entity'].startswith("I") and rec['entity'].split("-")[-1] == current_type:
      current_end += 1
      if rec['word'].startswith("#") or rec['word'] in punctuation:
        current += rec['word'].strip("#")
      else:
        current += " " + rec['word']
    else:
      pass

  if current:
    entities.append({"word": current, "entity": current_type, "start": current_start, "end": current_end})

  return entities

In [24]:
full_entities = recompose_entities(ents)
full_entities

[{'word': 'Gibilterra', 'entity': 'LOC', 'start': 1, 'end': 4},
 {'word': 'Gibraltar', 'entity': 'LOC', 'start': 9, 'end': 10},
 {'word': 'Regno Unito', 'entity': 'LOC', 'start': 40, 'end': 42},
 {'word': 'Nazioni Unite', 'entity': 'ORG', 'start': 52, 'end': 55},
 {'word': 'Europa', 'entity': 'LOC', 'start': 65, 'end': 66},
 {'word': 'penisola Iberica', 'entity': 'LOC', 'start': 75, 'end': 79},
 {'word': 'oceano Atlantico', 'entity': 'LOC', 'start': 92, 'end': 96},
 {'word': 'mar Mediterraneo', 'entity': 'LOC', 'start': 98, 'end': 100},
 {'word': 'La Línea de la Concepción',
  'entity': 'LOC',
  'start': 107,
  'end': 112},
 {'word': 'provincia di Cadice', 'entity': 'LOC', 'start': 116, 'end': 120},
 {'word': 'Andalusia', 'entity': 'LOC', 'start': 121, 'end': 124},
 {'word': 'Gibilterra', 'entity': 'LOC', 'start': 126, 'end': 129},
 {'word': 'Spagna', 'entity': 'LOC', 'start': 132, 'end': 133},
 {'word': 'Schengen', 'entity': 'LOC', 'start': 149, 'end': 152},
 {'word': 'Calpe', 'entity

In [25]:
!pip install mbrs &> /dev/null

In [26]:
from mbrs.metrics import Metric
from mbrs.decoders import DecoderMBR
from random import sample

SOURCE = ex
HYPOTHESES = ["demonym", "named after", "native label", "official name", "image", "inception", "instance of"]
N_PSEUDOREF = 8

In [27]:
selected_pair = sample(full_entities, 2)
selected_pair

[{'word': 'Calpe', 'entity': 'LOC', 'start': 160, 'end': 162},
 {'word': 'Jabal Ţāriq', 'entity': 'LOC', 'start': 183, 'end': 187}]

In [28]:
import torch
from typing import List, Optional
from torch.func import jacrev

class OutputOnlyModel(torch.nn.Module):

    def __init__(self, model: torch.nn.Module):
        super().__init__()

        self.model = model

    def forward(self, X: torch.Tensor, pred_id: List[int] = None, select: torch.Tensor = None, attention_mask = None):
        if hasattr(self.model, "bert"):
            if attention_mask is not None:
                attention_mask = attention_mask.reshape(X.shape[0], X.shape[1], 1)
            encoder_output = self.model.bert.encoder(X, attention_mask = attention_mask)['last_hidden_state']

            if hasattr(self.model, "pooler"):
                encoder_output = self.model.bert.pooler(encoder_output)

            if hasattr(self.model, "classifier"):
                y = self.model.classifier(
                    encoder_output
                )

            elif hasattr(self.model, "cls"):
                y = self.model.cls(
                    encoder_output[:,pred_id,:]
                )

            else:
                raise AttributeError("BERT model has neither classifier nor cls head")
        else:
            encoder_output, _ = self.model.encoder(X)
            y = self.model.classifier(encoder_output[:, 0])

        if select is not None:
            select = select.unsqueeze(0)
            y = torch.gather(
                y,
                dim = -1,
                index = select[:,pred_id,:]
            )

        return y, torch.argmax(y, dim = -1)

def jacobian(nn_input: torch.Tensor, model: torch.nn.Module = None, select: torch.Tensor = None, pred_id: List[int] = None, attention_mask = None):
    """
    Computes the full Jacobian matrix of the neural network output with respect
    to its input.

    Args:
        nn_output (torch.Tensor): The output tensor of a neural network where
        each element depends on the input tensor and has gradients enabled.
        nn_input (torch.Tensor): The input tensor to the neural network with
        gradients enabled.

    Returns:
        torch.Tensor: A tensor representing the Jacobian matrix. The dimensions
        of the matrix will be [len(nn_output), len(nn_input)], reflecting the
        partial derivatives of each output element with respect to each input element.
    """

    if pred_id is None:
        pred_id = [0]*nn_input.shape[0]

    if select is None:
        select = [None]*nn_input.shape[0]

    # nn_input.shape = (batch_size, N_patches+1, embedding_size)
    batch_jacobian, predictions = list(zip(*[
        jacrev(model, argnums = 0, has_aux=True)(
            x,
            pred_id[i],
            select[i],
            None if attention_mask is None else attention_mask[i]
        )
    for i,x in enumerate(nn_input.split(1))]))
    batch_jacobian = torch.stack(batch_jacobian)
    predictions = torch.stack(predictions)
    # batch_jacobian.shape = (batch_size, output_size, (N_patches+1), embedding_size)

    batch_jacobian = torch.squeeze(batch_jacobian, dim = list(range(1,batch_jacobian.dim())))
    predictions = torch.squeeze(predictions, dim = list(range(1,predictions.dim())))

    return batch_jacobian, predictions


def pullback(
    input_simec: torch.Tensor,
    g: torch.Tensor,
    model: torch.nn.Module = None,
    eq_class_emb_ids: List[List[int]] = None,
    select: torch.Tensor = None,
    pred_id: List[int] = None,
    degrowth: bool = True,
    same_equivalence_class: bool = True,
    attention_mask = None,
):
    """
    Computes the pullback metric tensor using the given input and output embeddings and a metric tensor g.

    Args:
        input_simec (torch.Tensor): Input embeddings tensor.
        output_simec (torch.Tensor): Output embeddings tensor derived from the
        input embeddings.
        g (torch.Tensor): Metric tensor g used as the Riemannian metric in the
        output space, of size (batch_size, embedding_size, embedding_size).
        eq_class_emb_ids (List[int], optional): Indices of embeddings to be
        considered for the pullback. If provided, restricts the computation to
        these embeddings.
        model (torch.nn.Module): Model to compute the Jacobian of.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Eigenvalues and eigenvectors of the
        pullback metric tensor.
    """

    if pred_id is None:
        pred_id = [0]*input_simec.shape[0]

    jac, predictions = jacobian(input_simec, model, select=select, pred_id = pred_id, attention_mask = attention_mask)
    # jac.shape = (batch_size, output_size, N_patches+1, embedding_size)
    # predictions.shape = (batch_size,)
    while jac.dim() < 4:
        jac = torch.unsqueeze(jac, 0)

    # Select ids and pad if necessary
    max_len = max(map(len, eq_class_emb_ids))
    batch_size = jac.shape[0]
    jac = torch.stack(
        [
            torch.nn.functional.pad(jac[i,:,L,:], (0,0,0,max_len-len(L)))
        for i,L in enumerate(eq_class_emb_ids)]
    )
    # jac.shape = (batch_size, output_size, max_len, embedding_size)

    with torch.no_grad():
        jac = torch.transpose(jac, 1, 2)
        jac = torch.flatten(jac, end_dim = 1)
        # jac.shape = (batch_size*max_len, output_size, embedding_size)
        jac_t = torch.transpose(jac, -2, -1)
        # jac_t.shape = (batch_size*max_len, embedding_size, output_size)

        # g.shape = (batch_size, max_len, output_size, output_size)
        tmp = torch.bmm(jac_t, torch.flatten(g, end_dim=1))
        # tmp.shape = (batch_size*max_len, embedding_size, output_size)
        pullback_metric = torch.bmm(tmp, jac)
        # pullback_metric.shape = (batch_size*max_len, embedding_size, embedding_size)

        pullback_metric = torch.stack(
            torch.chunk(pullback_metric, batch_size)
        )
        # pullback_metric.shape = (batch_size, max_len, embedding_size, embedding_size)

        eigenvalues, eigenvectors = torch.linalg.eigh(pullback_metric)
        # eigenvalues.shape = (batch_size, max_len, embedding_size)
        # eigenvectors.shape = (batch_size, max_len, embedding_size, embedding_size)

        if degrowth and same_equivalence_class:
            jac_eigen_dot_prod = torch.bmm(jac, torch.flatten(eigenvectors, end_dim=1))
            # jac_eigen_dot_prod.shape = (batch_size*max_len, output_size, embedding_size)
            jac_eigen_dot_prod = torch.gather(jac_eigen_dot_prod, dim = 1, index = predictions.repeat_interleave(max_len).unsqueeze(-1).repeat(1,1,eigenvectors.shape[-1]))
            jac_eigen_dot_prod = torch.stack(
                torch.chunk(jac_eigen_dot_prod.transpose(0,1), batch_size)
            )
            jac_eigen_dot_prod = torch.squeeze(jac_eigen_dot_prod, dim = list(range(1,jac_eigen_dot_prod.dim()))).reshape(eigenvalues.shape)
            # jac_eigen_dot_prod.shape = (batch_size, max_len, embedding_size)
            eigenvalues = torch.where(jac_eigen_dot_prod < 0, eigenvalues, torch.zeros_like(eigenvalues))

    return eigenvalues, eigenvectors

In [29]:
tokenized_pair = tokenizer([sp['word'] for sp in selected_pair], padding = True, return_tensors = "pt", add_special_tokens = True)
embedded_pair = model.bert.embeddings(tokenized_pair.input_ids)
embedded_pair.shape, tokenized_pair.attention_mask.shape

(torch.Size([2, 6, 768]), torch.Size([2, 6]))

In [30]:
g = (
        torch.eye(len(labelset))
        .unsqueeze(0)
        .unsqueeze(0)
        .repeat(
            embedded_pair.shape[0],
            embedded_pair.shape[1],
            1,
            1,
        )
    )

eq_class_emb_ids = [list(range(mask.sum())) for mask in tokenized_pair.attention_mask]

max_len = max(map(len, eq_class_emb_ids))
g = torch.stack(
    [
        torch.nn.functional.pad(g[i,L,:,:], (0,0,0,0,0,max_len-len(L)))
    for i,L in enumerate(eq_class_emb_ids)]
)

select = torch.nn.utils.rnn.pad_sequence([y.logits[0][sp['start']:sp['end']].argmax(dim = -1).unique() for sp in selected_pair])
select = select.unsqueeze(1).expand(-1, max_len, -1)
pred_id = torch.as_tensor([0, 0]).unsqueeze(-1)

g.shape, eq_class_emb_ids, select.shape, pred_id

(torch.Size([2, 6, 9, 9]),
 [[0, 1, 2, 3], [0, 1, 2, 3, 4, 5]],
 torch.Size([2, 6, 2]),
 tensor([[0],
         [0]]))

In [31]:
g = torch.gather(g, dim = -1, index = select.unsqueeze(-1).expand(-1,-1,-1,select.shape[-1]))
g.shape

torch.Size([2, 6, 2, 2])

In [32]:
eigenvals, eigenvecs = pullback(embedded_pair, g = g, model = OutputOnlyModel(model), eq_class_emb_ids = eq_class_emb_ids, select = select, pred_id = pred_id, attention_mask = tokenized_pair.attention_mask.to(dtype = torch.bool))

In [33]:
number_eigenvalues = torch.sum(torch.abs(eigenvals) < 1e-6, dim=-1).float().unsqueeze(-1).expand(-1,-1,N_PSEUDOREF)
# Positions of the eigenvalues to choose (note they are in ascending order)
ids_eigenvals = torch.round(torch.rand_like(number_eigenvalues) * number_eigenvalues).to(dtype = torch.int64).unsqueeze(-1) # equal to randint for each element


In [34]:
number_eigenvalues.shape, eigenvals.shape

(torch.Size([2, 6, 8]), torch.Size([2, 6, 768]))

In [35]:
min_nonzero_eigenvals = torch.gather(eigenvals, dim = -1, index = number_eigenvalues[...,[1]].to(dtype = torch.int32) - 1)
max_eigenvals = eigenvals[..., [-1]] + 1e-6
inv_root_condition_numbers = torch.sqrt(torch.abs(min_nonzero_eigenvals / max_eigenvals)) + 1e-3 * torch.randn_like(max_eigenvals)
inv_root_condition_numbers

tensor([[[ 1.2119e-03],
         [ 2.2310e-03],
         [-1.6950e-03],
         [-1.6013e-04],
         [ 4.7143e-04],
         [-6.6407e-04]],

        [[-2.4817e-04],
         [-5.5308e-04],
         [ 1.3685e-04],
         [ 3.0275e-05],
         [-1.7336e-03],
         [-2.0248e-03]]])

In [36]:
# Choice of eigenvalues and eigenvectors
ids_eigenvals = torch.clamp(ids_eigenvals, 0, eigenvals.shape[2]-1)
selected_eigenvecs = torch.gather(eigenvecs, dim = -2, index = ids_eigenvals.expand(-1,-1,-1,eigenvecs.shape[-1]))

In [37]:
input_embeddings = embedded_pair.unsqueeze(2) + inv_root_condition_numbers.unsqueeze(-1) * selected_eigenvecs
input_embeddings.shape

torch.Size([2, 6, 8, 768])

In [38]:
from itertools import product as cartesian
references = list(cartesian(*[torch.chunk(emb, chunks = input_embeddings.shape[-2], dim = -2) for emb in input_embeddings]))
len(references)

64

In [39]:
!pip install torchdr &> /dev/null
!pip install POT &> /dev/null

In [53]:
from dataclasses import dataclass
from torchdr import KernelPCA, GaussianAffinity
from tqdm.auto import tqdm
from scipy.spatial.distance import cdist, sqeuclidean
from transformers import AutoModel
from ot.unbalanced import sinkhorn_stabilized_unbalanced
import numpy as np

class CustomMetric(Metric):

  @dataclass
  class Config(Metric.Config):
    model_name: str = None
    embedding_module_name: str = "embeddings"
    encoder_module_name: str = "encoder"

  def __init__(self, cfg):
    super().__init__(cfg)

    self.tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
    self.model = AutoModel.from_pretrained(cfg.model_name)
    self.embedder = getattr(self.model, cfg.embedding_module_name)
    self.encoder = getattr(self.model, cfg.encoder_module_name)
    self.n_neighbors = 15
    self.manifold = ParametricUMAP(
        n_neighbors = self.n_neighbors,
        hidden_dim = self.model.config.hidden_size * 3 // 4,
        n_components=self.model.config.hidden_size // 2,
        n_layers = self.model.config.max_position_embeddings // self.model.config.hidden_size * self.n_neighbors + 1,
        verbose = False
    )
    self.alignment = sinkhorn_stabilized_unbalanced
    self.current_source = ""

  def score(
        self, hypothesis: str, references: List[torch.Tensor], source: Optional[str] = None
    ) -> float:
        """Calculate the score of the given hypothesis.
        """
        references = [torch.squeeze(ref) for ref in references]
        tokenized_source_and_hp = self.tokenizer([source, hypothesis], padding = True, add_special_tokens = False, return_tensors = "pt")
        with torch.inference_mode():
          embedded_source_and_hp = self.embedder(tokenized_source_and_hp.input_ids)

          embedded_triple = torch.cat([references[0], embedded_source_and_hp[1][:tokenized_source_and_hp.attention_mask[1].sum()], references[1]], dim = 0)
          embedded_source = embedded_source_and_hp[0]

          padded_input = torch.nn.utils.rnn.pad_sequence([embedded_triple, embedded_source, embedded_triple[:-references[1].shape[0]]], batch_first = True)
          output = self.encoder(padded_input).last_hidden_state

        output = output.detach().clone()

        # First, learn the manifold of the source space
        if source != self.current_source:
          self.current_source = source
          self.manifold.fit(output[1])

        manifold_triple = self.manifold.transform(output[0][:embedded_triple.shape[0]])
        manifold_source = self.manifold.transform(output[1])
        manifold_head_rel = self.manifold.transform(output[2][:embedded_triple.shape[0]])

        # Now I want to align the first and the second row in a wise manner
        # Alignment 1: source <-> (ref1, hp, ref2)
        pair_dists = cdist(np.concatenate([manifold_source, manifold_head_rel]), manifold_triple, metric = sqeuclidean)
        pair_dists = pair_dists / pair_dists.max()
        A1 = self.alignment(a = [], b = [], M = pair_dists[:manifold_source.shape[0]], reg = 1.0, reg_m = (0.5, 2.0))
        d1 = (pair_dists[:manifold_source.shape[0]] * A1).sum()

        # Alignment 2: (ref1, hp, [MASK]) <-> (ref1, hp, ref2)
        A2 = self.alignment(a = [], b = [], M = pair_dists[manifold_source.shape[0]:], reg = 0.8, reg_m = (1.0, 1.0))
        d2 = (pair_dists[manifold_source.shape[0]:] * A2).sum()

        return ((d1 + 1e-6)**(-1) + (d2 + 1e-6)**(-1)) / 2


In [54]:
metric_cfg = CustomMetric.Config(
  model_name=model_name
)
metric = CustomMetric(metric_cfg)

In [55]:
# Setup MBR decoding.
decoder_cfg = DecoderMBR.Config()
decoder = DecoderMBR(decoder_cfg, metric)
pseudorefs = tqdm(references)
candidates = [tokenizer.special_tokens_map['mask_token'] + " " + p + " " + tokenizer.special_tokens_map['mask_token'] for p in HYPOTHESES]

output = decoder.decode(candidates, pseudorefs, source=SOURCE, nbest=1)

print(f"Selected index: {output.idx}")
print(f"Output sentence: {selected_pair[0]['word']} {output.sentence} {selected_pair[1]['word']}")
print(f"Expected score: {output.score}")

  0%|          | 0/64 [00:00<?, ?it/s]

Sampling negative edges...


Completed processes:   0%|          | 0/2 [00:00<?, ?it/s]

Training...


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 9/10, Loss: 0.0497


Epoch 10:   0%|          | 0/733 [00:00<?, ?it/s]

Epoch 10/10, Loss: 0.0483


/usr/local/lib/python3.12/dist-packages/parametric_umap/core.py:268: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



Selected index: [2]
Output sentence: Calpe ['[MASK] native label [MASK]'] Jabal Ţāriq
Expected score: [138.1943817138672]
